# Calculating ENSO Using Intake-ESGF

## Overview

In this workflow, we combine topics covered in previous Pythia Foundations and CMIP6 Cookbook content to apply the [Niño 3.4 Index](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni) to a broader range of datasets. As a refresher of what the ENSO 3.4 index is, please see the following text, which is also included in the [ENSO Xarray](https://foundations.projectpythia.org/core/xarray/enso-xarray.html) content in the Pythia Foundations content.

> Niño 3.4 (5N-5S, 170W-120W): The Niño 3.4 anomalies may be thought of as representing the average equatorial SSTs across the Pacific from about the dateline to the South American coast. The Niño 3.4 index typically uses a 5-month running mean, and El Niño or La Niña events are defined when the Niño 3.4 SSTs exceed +/- 0.4C for a period of six months or more.

> Niño X Index computation: a) Compute area averaged total SST from Niño X region; b) Compute monthly climatology (e.g., 1950-1979) for area averaged total SST from Niño X region, and subtract climatology from area averaged total SST time series to obtain anomalies; c) Smooth the anomalies with a 5-month running mean; d) Normalize the smoothed values by its standard deviation over the climatological period.

![](https://www.ncdc.noaa.gov/monitoring-content/teleconnections/nino-regions.gif)

The previous example in the Pythia Foundations content detailed a single simulation. In this example, we aim to apply this computation more generically across a variety of datasets.

The overall goal of this tutorial is to produce a plot of ENSO data using Xarray and intake-ESGF. The plots will resemble the Oceanic Niño Index plot shown below.

![ONI index plot from NCAR Climate Data Guide](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray/xarray-intro.html) | Necessary | |
| [hvPlot Basics](https://hvplot.holoviz.org/getting_started/hvplot.html) | Necessary | Interactive Visualization with hvPlot |
| [Understanding of NetCDF](https://foundations.projectpythia.org/core/data-formats/netcdf-cf.html) | Helpful | Familiarity with metadata structure |
| [Calculating ENSO with Xarray](https://foundations.projectpythia.org/core/xarray/enso-xarray.html) | Neccessary | Understanding of Masking and Xarray Functions |
| Dask | Helpful | |

- **Time to learn**: 30 minutes

## Imports

In [1]:
import hvplot.xarray
import holoviews as hv
import numpy as np
import hvplot.xarray
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from intake_esgf import ESGFCatalog
import xarray as xr
import cf_xarray
import warnings
warnings.filterwarnings("ignore")

hv.extension("bokeh")

## Access ESGF-hosted CMIP6 Data
We will use the Climate Model Intercomparison Project version 6 (CMIP6) dataset, which is available from the Earth System Grid Federation (ESGF) data servers.

There is a toolkit, `intake-esgf`, we can use to interface with the data servers, making it easier to search for our datasets.

In [2]:
cat = ESGFCatalog()
cat.search(
        activity_id='CMIP',
        experiment_id=["historical","ssp585"],
        source_id="CESM2",
        variable_id=["tos"],
        member_id='r11i1p1f1',
        table_id="Omon",
    )

   Searching indices:   0%|          |0/1 [       ?index/s]

   Searching indices: 100%|##########|1/1 [    1.01index/s]

   Searching indices: 100%|##########|1/1 [    1.01index/s]

Summary information for 2 results:
mip_era                [CMIP6]
activity_id             [CMIP]
institution_id          [NCAR]
source_id              [CESM2]
experiment_id     [historical]
member_id          [r11i1p1f1]
table_id                [Omon]
variable_id              [tos]
grid_label            [gn, gr]
dtype: object

### Load into a DataTree
Once we subset for our data, we can load the data into a datatree, which is a nested structure of `xarray.Dataset`s, which include the climate grid cell statistics as well!

In [3]:
tos_tree = cat.to_datatree()
tos_tree

 Obtaining file info:   0%|          |0/2 [     ?dataset/s]

 Obtaining file info:  50%|#####     |1/2 [  1.10dataset/s]

 Obtaining file info: 100%|##########|2/2 [  1.21dataset/s]

 Obtaining file info: 100%|##########|2/2 [  1.19dataset/s]

tos_Omon_CESM2_historical_r11i1p1f1_gn_185001-189912.nc:   0%|          |0.00/150M [?B/s]

...

areacello_Ofx_CESM2_historical_r11i1p1f1_gr.nc:  44%|####3     |247k/563k [665kB/s]

areacello_Ofx_CESM2_historical_r11i1p1f1_gr.nc: 100%|##########|563k/563k [1.16MB/s]

Adding cell measures: 100%|##########|2/2 [  6.86s/dataset]

Adding cell measures: 100%|##########|2/2 [  7.03s/dataset]

DataTree('None', parent=None)
├── DataTree('gn')
│       Dimensions:    (time: 1980, nlat: 384, nlon: 320, d2: 2, vertices: 4)
│       Coordinates:
│           lat        (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
│           lon        (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
│         * nlat       (nlat) int32 1 2 3 4 5 6 7 8 ... 377 378 379 380 381 382 383 384
│         * nlon       (nlon) int32 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
│         * time       (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 12:00:00
│       Dimensions without coordinates: d2, vertices
│       Data variables:
│           tos        (time, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
│           time_bnds  (time, d2) object dask.array<chunksize=(1, 2), meta=np.ndarray>
│           lat_bnds   (time, nlat, nlon, vertices) float32 dask.array<chunksize=(600, 384, 320, 4), meta=np.ndarray>
│           lon_bnds   (time, nlat, nlon, vertices) float32 dask.array<chunksize=(600, 384, 320, 4), meta=np.ndarray>
│           areacello  (nlat, nlon) float32 ...
│           sftof      (nlat, nlon) float32 ...
│       Attributes: (12/45)
│           Conventions:            CF-1.7 CMIP-6.2
│           activity_id:            CMIP
│           branch_method:          standard
│           branch_time_in_child:   674885.0
│           branch_time_in_parent:  219000.0
│           case_id:                972
│           ...                     ...
│           sub_experiment_id:      none
│           table_id:               Omon
│           tracking_id:            hdl:21.14100/b0ffb89d-095d-4533-a159-a2e1241ff138
│           variable_id:            tos
│           variant_info:           CMIP6 20th century experiments (1850-2014) with C...
│           variant_label:          r11i1p1f1
└── DataTree('gr')
        Dimensions:    (time: 1980, d2: 2, lat: 180, lon: 360)
        Coordinates:
          * time       (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 12:00:00
          * lat        (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
          * lon        (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
        Dimensions without coordinates: d2
        Data variables:
            time_bnds  (time, d2) object dask.array<chunksize=(1, 2), meta=np.ndarray>
            lat_bnds   (time, lat, d2) float64 dask.array<chunksize=(600, 180, 2), meta=np.ndarray>
            lon_bnds   (time, lon, d2) float64 dask.array<chunksize=(600, 360, 2), meta=np.ndarray>
            tos        (time, lat, lon) float32 dask.array<chunksize=(1, 180, 360), meta=np.ndarray>
            areacello  (lat, lon) float64 ...
        Attributes: (12/45)
            Conventions:            CF-1.7 CMIP-6.2
            activity_id:            CMIP
            branch_method:          standard
            branch_time_in_child:   674885.0
            branch_time_in_parent:  219000.0
            case_id:                972
            ...                     ...
            sub_experiment_id:      none
            table_id:               Omon
            tracking_id:            hdl:21.14100/0b412958-1dae-4684-9a1e-0c2ab29c492e
            variable_id:            tos
            variant_info:           CMIP6 20th century experiments (1850-2014) with C...
            variant_label:          r11i1p1f1

The nodes in the tree represent two different, grids. We would like to stay on the native model grid, using the `gn` node of the datatree, which represents the model native grid data.

In [4]:
ds = tos_tree["gn"].to_dataset()
ds

<xarray.Dataset>
Dimensions:    (time: 1980, nlat: 384, nlon: 320, d2: 2, vertices: 4)
Coordinates:
    lat        (nlat, nlon) float64 -79.22 -79.22 -79.22 ... 72.2 72.19 72.19
    lon        (nlat, nlon) float64 320.6 321.7 322.8 ... 318.9 319.4 319.8
  * nlat       (nlat) int32 1 2 3 4 5 6 7 8 ... 377 378 379 380 381 382 383 384
  * nlon       (nlon) int32 1 2 3 4 5 6 7 8 ... 313 314 315 316 317 318 319 320
  * time       (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 12:00:00
Dimensions without coordinates: d2, vertices
Data variables:
    tos        (time, nlat, nlon) float32 dask.array<chunksize=(1, 384, 320), meta=np.ndarray>
    time_bnds  (time, d2) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (time, nlat, nlon, vertices) float32 dask.array<chunksize=(600, 384, 320, 4), meta=np.ndarray>
    lon_bnds   (time, nlat, nlon, vertices) float32 dask.array<chunksize=(600, 384, 320, 4), meta=np.ndarray>
    areacello  (nlat, nlon) float32 ...
    sftof      (nlat, nlon) float32 ...
Attributes: (12/45)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            CMIP
    branch_method:          standard
    branch_time_in_child:   674885.0
    branch_time_in_parent:  219000.0
    case_id:                972
    ...                     ...
    sub_experiment_id:      none
    table_id:               Omon
    tracking_id:            hdl:21.14100/b0ffb89d-095d-4533-a159-a2e1241ff138
    variable_id:            tos
    variant_info:           CMIP6 20th century experiments (1850-2014) with C...
    variant_label:          r11i1p1f1

## Calculate ENSO
The calculation is covered in more detail in the Pythia Foundations book, here, we apply the calculation to our datasets!

In [5]:
def calculate_enso(ds):
    
    # Subset the El Nino 3.4 index region
    dso = ds.where(
    (ds.cf["latitude"] < 5) & (ds.cf["latitude"] > -5) & (ds.cf["longitude"] > 190) & (ds.cf["longitude"] < 240), drop=True
    )
    
    # Calculate the monthly means
    gb = dso.tos.groupby('time.month')
    
    # Subtract the monthly averages, returning the anomalies
    tos_nino34_anom = gb - gb.mean(dim='time')
    
    # Determine the non-time dimensions and average using these
    non_time_dims = set(tos_nino34_anom.dims)
    non_time_dims.remove(ds.tos.cf["T"].name)
    weighted_average = tos_nino34_anom.weighted(ds["areacello"]).mean(dim=list(non_time_dims))
    
    # Calculate the rolling average
    rolling_average = weighted_average.rolling(time=5, center=True).mean()
    std_dev = weighted_average.std()
    return rolling_average / std_dev

In [6]:
enso_index = calculate_enso(ds).compute()
enso_index

<xarray.DataArray 'tos' (time: 1980)>
array([       nan,        nan, 0.06341499, ..., 0.79205155,        nan,
              nan], dtype=float32)
Coordinates:
  * time     (time) object 1850-01-15 13:00:00.000007 ... 2014-12-15 12:00:00
    month    (time) int64 1 2 3 4 5 6 7 8 9 10 11 ... 2 3 4 5 6 7 8 9 10 11 12

## Visualize ENSO

### Basic Visualization
We can create a basic visualization of the dataset using hvplot!

In [7]:
enso_index.hvplot(x='time')

:Curve   [time]   (tos)

### Identify El Niño and La Niña
Including the indices as we showed above is not always the most helpful. We need to add additional context to help the reader understand when we reach El Niño and La Niña, which are helpful thresholds for the wider community to use.

A typical threshold to use is 0.4, which means El Niño occurs when the ENSO 3.4 index is equal to or greater than 0.4, and La Niña occurs when the ENSO 3.4 index is equal to or less than 0.4.

We apply this using the following function.

In [8]:
def add_enso_thresholds(da, threshold=0.4):
    
    # Conver the xr.DataArray into an xr.Dataset
    ds = da.to_dataset()
    
    # Cleanup the time and use the thresholds
    try:
        ds["time"]= ds.indexes["time"].to_datetimeindex()
    except:
        pass
    ds["tos_gt_04"] = ("time", ds.tos.where(ds.tos >= threshold, threshold).data)
    ds["tos_lt_04"] = ("time", ds.tos.where(ds.tos <= -threshold, -threshold).data)
    
    # Add fields for the thresholds
    ds["el_nino_threshold"] = ("time", np.zeros_like(ds.tos) + threshold)
    ds["la_nina_threshold"] = ("time", np.zeros_like(ds.tos) - threshold)
    
    return ds

In [9]:
enso_ds = add_enso_thresholds(enso_index)
enso_ds

<xarray.Dataset>
Dimensions:            (time: 1980)
Coordinates:
  * time               (time) datetime64[ns] 1850-01-15T13:00:00.000007 ... 2...
    month              (time) int64 1 2 3 4 5 6 7 8 9 ... 4 5 6 7 8 9 10 11 12
Data variables:
    tos                (time) float32 nan nan 0.06341 ... 0.7921 nan nan
    tos_gt_04          (time) float32 0.4 0.4 0.4 0.4 ... 0.6829 0.7921 0.4 0.4
    tos_lt_04          (time) float32 -0.4 -0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4
    el_nino_threshold  (time) float32 0.4 0.4 0.4 0.4 0.4 ... 0.4 0.4 0.4 0.4
    la_nina_threshold  (time) float32 -0.4 -0.4 -0.4 -0.4 ... -0.4 -0.4 -0.4

### Configure a Function to Plot the Data
We will use the `hvplot.area` functionality here, which enables us to shade the area between values. We use the newly added variables in our dataset to help here.

In [10]:
def plot_enso(ds):
    el_nino = ds.hvplot.area(x="time", y2='tos_gt_04', y='el_nino_threshold', color='red', hover=False)
    el_nino_label = hv.Text(ds.isel(time=40).time.values, 2, 'El Niño').opts(text_color='red',)

    # Create the La Niña area graphs
    la_nina = ds.hvplot.area(x="time", y2='tos_lt_04', y='la_nina_threshold', color='blue', hover=False)
    la_nina_label = hv.Text(ds.isel(time=-40).time.values, -2, 'La Niña').opts(text_color='blue')

    # Plot a timeseries of the ENSO 3.4 index
    enso = ds.tos.hvplot(x='time', line_width=0.5, color='k', xlabel='Year', ylabel='ENSO 3.4 Index')

    # Combine all the plots into a single plot
    return (el_nino_label * la_nina_label * el_nino * la_nina * enso)

In [11]:
plot_enso(enso_ds)

:Overlay
   .Text.I  :Text   [x,y]
   .Text.II :Text   [x,y]
   .Area.I  :Area   [time]   (el_nino_threshold,tos_gt_04)
   .Area.II :Area   [time]   (la_nina_threshold,tos_lt_04)
   .Curve.I :Curve   [time]   (tos)

## Apply to Multiple Datasets
Now that we have the workflow, let's apply this to multiple datasets. We focus here on two different instiutions:
- The National Center for Atmospheric Research (NCAR)
- Model for Interdisciplinary Research on Climate (MIROC)

Both of these modeling centers produced output for CMIP6.


### Setup a Function for Searching and Combining Datasets
We can use the query mentioned previously to configure our search. Here, we parameterize based on the institution id (ex. NCAR, MIROC).

In [12]:
def search_esgf(institution_id, grid='gn'):
    
    # Search and load the ocean surface temperature (tos)
    cat = ESGFCatalog()
    cat.search(
        activity_id="CMIP",
        experiment_id="historical",
        institution_id=institution_id,
        variable_id=["tos"],
        member_id='r11i1p1f1',
        table_id="Omon",
    )
    try:
        tos_ds = cat.to_datatree()[grid].to_dataset()
    except KeyError:
        tos_ds = cat.to_dataset_dict()["tos"]

    return tos_ds

### Apply the Search and Computations

In [13]:
ncar_ds = search_esgf("NCAR")
enso_index_ncar = add_enso_thresholds(calculate_enso(ncar_ds).compute())

   Searching indices:   0%|          |0/1 [       ?index/s]

   Searching indices: 100%|##########|1/1 [    1.13s/index]

   Searching indices: 100%|##########|1/1 [    1.13s/index]

 Obtaining file info:   0%|          |0/2 [     ?dataset/s]

 Obtaining file info:  50%|#####     |1/2 [  1.62dataset/s]

...

 Obtaining file info: 100%|##########|2/2 [  1.43dataset/s]

Adding cell measures:   0%|          |0/2 [     ?dataset/s]

Adding cell measures:  50%|#####     |1/2 [  3.50s/dataset]

Adding cell measures: 100%|##########|2/2 [  4.98s/dataset]

Adding cell measures: 100%|##########|2/2 [  4.76s/dataset]

In [14]:
miroc_ds = search_esgf("MIROC")
enso_index_miroc = add_enso_thresholds(calculate_enso(miroc_ds).compute())

   Searching indices:   0%|          |0/1 [       ?index/s]

   Searching indices: 100%|##########|1/1 [    1.35index/s]

   Searching indices: 100%|##########|1/1 [    1.35index/s]

 Obtaining file info:   0%|          |0/1 [     ?dataset/s]

 Obtaining file info: 100%|##########|1/1 [  1.54dataset/s]

...

 Obtaining file info: 100%|##########|1/1 [  1.20dataset/s]

 Obtaining file info: 100%|##########|1/1 [  1.19dataset/s]

Adding cell measures:   0%|          |0/1 [     ?dataset/s]

Adding cell measures: 100%|##########|1/1 [  3.26s/dataset]

Adding cell measures: 100%|##########|1/1 [  3.26s/dataset]

### Visualize our ENSO Comparison
Now that we have our data, we can plot the comparison, stacking the two together using hvPlot.

In [15]:
ncar_enso_plot = plot_enso(enso_index_ncar).opts(title=f'NCAR {ncar_ds.attrs["source_id"]} \n Ensemble Member: {ncar_ds.attrs["variant_label"]}')
miroc_enso_plot = plot_enso(enso_index_miroc).opts(title=f'MIROC {miroc_ds.attrs["source_id"]} \n Ensemble Member: {miroc_ds.attrs["variant_label"]}')

(ncar_enso_plot + miroc_enso_plot).cols(1)

:Layout
   .Overlay.I  :Overlay
      .Text.I  :Text   [x,y]
      .Text.II :Text   [x,y]
      .Area.I  :Area   [time]   (el_nino_threshold,tos_gt_04)
      .Area.II :Area   [time]   (la_nina_threshold,tos_lt_04)
      .Curve.I :Curve   [time]   (tos)
   .Overlay.II :Overlay
      .Text.I  :Text   [x,y]
      .Text.II :Text   [x,y]
      .Area.I  :Area   [time]   (el_nino_threshold,tos_gt_04)
      .Area.II :Area   [time]   (la_nina_threshold,tos_lt_04)
      .Curve.I :Curve   [time]   (tos)

## Summary
In this notebook, we searched for and accessed two different CMIP6 datasets hosted through ESGF, calculated the ENSO 3.4 indices for the datasets, and created interactive plots comparing where we see El Niño and La Niña.

### What's next?
We will see some more advanced examples of using the CMIP6 and other data access methods as well as computations

## Resources and references
- [Intake-ESGF Documentation](https://github.com/nocollier/intake-esgf)